# POCs su uso dei dati CPSV della PAT

La PAT pubblica l'elenco dei suoi servizi, online e no, in formato interoperabile(JSON)
https://www.provincia.tn.it/api/openapi/servizi

Sfruttando qualche tipo di "AI" cosa possiamo fare?


L'intelligenza artificiale (AI) può essere utilizzata in diversi modi per sfruttare i dati sui servizi pubblici strutturati secondo lo standard CPSV-AP (Core Public Service Vocabulary Application Profile) nella Provincia di Trento. Ecco alcune soluzioni pratiche che potrebbero essere implementate:

1. Chatbot AI per i Servizi ai Cittadini
Un chatbot alimentato dall'AI può essere il primo punto di contatto per i cittadini, rispondendo alle domande e guidandoli attraverso i servizi pubblici disponibili. Può:

Capire le domande dei cittadini grazie all'elaborazione del linguaggio naturale (NLP).
Associare le richieste ai servizi descritti nei dati CPSV-AP.
Fornire raccomandazioni personalizzate, indirizzando gli utenti verso specifici servizi o procedure.
Offrire supporto multilingue per diverse comunità.
Esempio: Un cittadino chiede come registrare un veicolo. Il chatbot potrebbe indicare il dipartimento giusto, fornire una guida passo-passo e, se disponibile, collegare l'utente al servizio online.

2. Ricerca Avanzata e Recupero di Informazioni
L'AI può migliorare la ricerca nei dati dei servizi pubblici:

Migliorare la rilevanza dei risultati di ricerca apprendendo dalle interazioni degli utenti.
Implementare una ricerca semantica che comprende il significato delle domande anziché limitarsi alle parole chiave.
Offrire filtri contestuali (ad esempio, per area geografica o disponibilità del servizio).
Esempio: Un utente cerca servizi per la "registrazione di un'azienda" in una specifica zona, e l'AI fornisce risultati personalizzati e indicazioni.

3. Sintesi e Panoramiche sui Servizi
Con più di 1.000 servizi pubblici, può essere difficile orientarsi. Strumenti di sintesi basati sull'AI possono:

Riassumere automaticamente le descrizioni dei servizi in formati più semplici.
Creare sintesi personalizzate in base alle esigenze dell'utente (ad esempio, riepiloghi brevi o elenchi puntati).
Esempio: Un utente interessato ad avviare un'attività potrebbe ricevere un riepilogo conciso di tutti i servizi rilevanti (permessi, registrazione fiscale, ecc.).

4. Visualizzazioni e Dashboard
L'AI può aiutare a creare visualizzazioni interattive e dashboard che aiutano a comprendere meglio i servizi pubblici:

Visualizzazioni geospaziali che mostrano la distribuzione dei servizi sul territorio.
Dashboard interattive filtrabili in base alle preferenze dell'utente.
Utilizzare analisi predittive per suggerire servizi basati su tendenze o servizi popolari.
Esempio: Una dashboard che mostra quali servizi sono più utilizzati in una specifica area o gruppo demografico.

5. Analisi Predittiva e Prescrittiva
L'AI può analizzare l'uso dei servizi pubblici e prevedere i bisogni futuri, aiutando l'amministrazione a:

Ottimizzare i servizi prevedendo quelli che potrebbero essere più richiesti.
Offrire suggerimenti personalizzati per i cittadini.
Notificare proattivamente gli utenti su scadenze o requisiti futuri (ad esempio, dichiarazioni fiscali, rinnovi di permessi).
Esempio: L'AI potrebbe notare che un utente interagisce spesso con servizi legati all'abitazione e suggerire altri servizi rilevanti come agevolazioni energetiche o tasse sulla proprietà.

6. Analisi del Sentimento e Feedback
L'AI può essere usata per analizzare i feedback e il sentimento dei cittadini riguardo ai servizi:

Raccogliere e analizzare feedback per capire dove ci sono problemi o insoddisfazioni.
Identificare eventuali lacune nei servizi offerti.
Fornire all'amministrazione suggerimenti per migliorare i servizi.
Esempio: L'AI potrebbe rilevare che un determinato servizio ha un basso indice di soddisfazione e suggerire come semplificarlo.

7. Motori di Raccomandazione
Sistemi di raccomandazione basati sull'AI possono suggerire ai cittadini servizi pertinenti in base alle loro interazioni o eventi della vita (come cambiamenti di residenza o nascita di un figlio).

Usare il filtraggio collaborativo per suggerire servizi basati su ciò che utenti simili hanno utilizzato.
Raccomandazioni contestuali attivate da eventi importanti nella vita degli utenti.
Esempio: Un utente che si è appena trasferito nella regione potrebbe ricevere informazioni su servizi come il cambio di residenza o l'iscrizione ai servizi sanitari locali.

8. Comprensione del Linguaggio Naturale per l'Analisi delle Descrizioni dei Servizi
Modelli AI possono analizzare continuamente le descrizioni dei servizi pubblici per:

Identificare ridondanze o incoerenze nelle descrizioni.
Suggerire miglioramenti per maggiore chiarezza e coerenza tra i servizi.
Automatizzare la categorizzazione dei servizi in gruppi tematici.
Conclusione
L'AI può migliorare notevolmente l'accessibilità, l'efficienza e la personalizzazione dei servizi pubblici nella Provincia di Trento sfruttando i dati strutturati secondo lo standard CPSV-AP. Soluzioni come chatbot, analisi predittive, ottimizzazione della ricerca e visualizzazioni possono migliorare l'esperienza degli utenti e facilitare l'interazione tra i cittadini e i servizi pubblici.

### TEST 1 - Un sistema di raccomandazione dei servizi in base a similarità testuale (non è semantico)

In [9]:
import requests
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Scarica la lista di stop words italiane (se non lo hai già fatto)
nltk.download('stopwords')
stop_words_italian = stopwords.words('italian')

# Step 1: Scarica i dati dal link fornito
url = 'https://www.provincia.tn.it/api/openapi/servizi'
response = requests.get(url)

# Step 2: Verifica il tipo di dati e carica il JSON
try:
    data = response.json()  # Converte la risposta in JSON
except json.JSONDecodeError:
    print("Errore nella decodifica del JSON")
    data = None

# Controlla che i dati siano nel formato atteso
if 'items' in data:
    services_data = data['items']
else:
    print("Formato JSON non valido. Impossibile trovare la chiave 'items'.")
    services_data = []

# Step 3: Estrarre le informazioni rilevanti (id, nome, descrizione)
services = []
for item in services_data:
    services.append({
        'id': item.get('id'),
        'name': item.get('name', 'Nome non disponibile').strip(),  # Rimuove eventuali spazi extra
        'description': item.get('description', 'Descrizione non disponibile')
    })

# Creare un DataFrame
df = pd.DataFrame(services)

# Step 4: Funzione di raccomandazione basata su matching parziale del testo inserito
def recommend_services(service_query='Contributo integrativo alla locazione sul libero mercato', top_n=5):
    # Step 4a: Normalizza il testo inserito dall'utente (lowercase e spazi rimossi)
    service_query = service_query.strip().lower()
    
    # Step 4b: Concatena nome e descrizione dei servizi per il matching
    df['combined_text'] = df['name'].str.lower() + " " + df['description'].str.lower()
    
    # Step 5: Calcolo della matrice TF-IDF sulla combinazione di nome e descrizione dei servizi
    tfidf = TfidfVectorizer(stop_words=stop_words_italian)  # Usa la lista di stop words italiane
    tfidf_matrix = tfidf.fit_transform(df['combined_text'])
    
    # Step 6: Calcolo della similarità del coseno tra l'input dell'utente e tutti i servizi
    user_input_tfidf = tfidf.transform([service_query])  # Trasforma il testo inserito dall'utente
    cosine_sim = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()  # Similarità con tutti i servizi
    
    # Step 7: Prendi i top N servizi ordinati per similarità
    sim_scores = list(enumerate(cosine_sim))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Ordina in base alla similarità
    sim_scores = sim_scores[:top_n]  # Prendi i top N
    
    # Recupera i nomi dei servizi raccomandati
    service_indices = [i[0] for i in sim_scores]
    recommended_services = df.iloc[service_indices]['name']
    
    return recommended_services

# Esempio di utilizzo: input dell'utente con valore predefinito
service_to_recommend = input("Inserisci un nome o parte del nome di un servizio (es. 'agricoltura'): ") or 'Contributo integrativo alla locazione sul libero mercato'
recommended = recommend_services(service_to_recommend, top_n=5)

# Mostra i risultati in un formato leggibile
print(f"Servizi simili a '{service_to_recommend}':")
for idx, service in enumerate(recommended, 1):
    print(f"{idx}. {service}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Inserisci un nome o parte del nome di un servizio (es. 'agricoltura'):  agricoltura


Servizi simili a 'agricoltura':
1. O.C.M. Vino - Promozione sui mercati dei Paesi terzi - campagna 2024/25
2. Abbonamento alle riviste "Il Trentino" e "Terra Trentina"
3. Contributi per irrigazione 2023
4. MONITORAGGIO DELL'UTILIZZO DI BIOMASSA
5. Contributo integrativo alla locazione sul libero mercato


### TEST 2 - Un sistema chatbot a supporto della ricerca

In [27]:
import requests
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

# Scarica la lista di stop words italiane (se non lo hai già fatto)
nltk.download('stopwords')
stop_words_italian = stopwords.words('italian')

# Step 1: Scarica i dati dal link fornito
url = 'https://www.provincia.tn.it/api/openapi/servizi'
response = requests.get(url)

# Step 2: Verifica il tipo di dati e carica il JSON
try:
    data = response.json()  # Converte la risposta in JSON
except json.JSONDecodeError:
    print("Errore nella decodifica del JSON")
    data = None

# Controlla che i dati siano nel formato atteso
if 'items' in data:
    services_data = data['items']
else:
    print("Formato JSON non valido. Impossibile trovare la chiave 'items'.")
    services_data = []

# Step 3: Estrarre le informazioni rilevanti (id, nome, descrizione, how_to, has_module_input)
services = []
for item in services_data:
    # Estrai i moduli dal campo 'has_module_input' come una lista di link
    modules = item.get('has_module_input', [])
    module_links = [module.get('uri', '') for module in modules if module.get('uri')]

    services.append({
        'id': item.get('id'),
        'name': item.get('name', 'Nome non disponibile').strip(),
        'description': item.get('description', 'Descrizione non disponibile'),
        'how_to': item.get('how_to', 'Informazioni non disponibili'),
        'module_links': module_links  # Lista di URI dei moduli
    })

# Creare un DataFrame
df = pd.DataFrame(services)

# Funzione per convertire l'HTML in plain text
def clean_html(raw_html):
    soup = BeautifulSoup(raw_html, 'html.parser')
    return soup.get_text()

# Step 4: Funzione di raccomandazione basata su matching parziale del testo inserito
def recommend_services(service_query='Contributo integrativo alla locazione sul libero mercato', top_n=5):
    # Step 4a: Normalizza il testo inserito dall'utente (lowercase e spazi rimossi)
    service_query = service_query.strip().lower()
    
    # Step 4b: Concatena nome e descrizione dei servizi per il matching
    df['combined_text'] = df['name'].str.lower() + " " + df['description'].str.lower()
    
    # Step 5: Calcolo della matrice TF-IDF sulla combinazione di nome e descrizione dei servizi
    tfidf = TfidfVectorizer(stop_words=stop_words_italian)  # Usa la lista di stop words italiane
    tfidf_matrix = tfidf.fit_transform(df['combined_text'])
    
    # Step 6: Calcolo della similarità del coseno tra l'input dell'utente e tutti i servizi
    user_input_tfidf = tfidf.transform([service_query])  # Trasforma il testo inserito dall'utente
    cosine_sim = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()  # Similarità con tutti i servizi
    
    # Step 7: Prendi i top N servizi ordinati per similarità
    sim_scores = list(enumerate(cosine_sim))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Ordina in base alla similarità
    sim_scores = sim_scores[:top_n]  # Prendi i top N
    
    # Recupera i nomi dei servizi raccomandati
    service_indices = [i[0] for i in sim_scores]
    recommended_services = df.iloc[service_indices][['name', 'description', 'how_to', 'module_links']]
    
    return recommended_services

# Funzione per simulare il chatbot
def chatbot():
    print("Chatbot: Benvenuto! Sono qui per aiutarti a trovare il servizio pubblico che stai cercando.")
    
    # Fase 1: Chiedi all'utente cosa sta cercando
    user_input = input("Chatbot: Per favore, dimmi cosa stai cercando (puoi inserire una parola chiave o una frase): ")
    
    # Fase 2: Trova servizi pertinenti
    recommended = recommend_services(user_input, top_n=5)
    
    # Fase 3: Mostra i risultati
    if recommended.empty:
        print(f"Chatbot: Non ho trovato servizi pertinenti per '{user_input}'. Prova a inserire un'altra parola chiave.")
    else:
        print(f"Chatbot: Ecco i servizi che ho trovato per '{user_input}':")
        for idx, row in enumerate(recommended.itertuples(), 1):
            print(f"{idx}. {row.name}")
        
        # Fase 4: Chiedi se l'utente vuole maggiori informazioni su uno dei servizi
        user_choice = input("Chatbot: Vuoi maggiori informazioni su uno di questi servizi? Inserisci il numero corrispondente oppure 'no': ").strip()
        
        if user_choice.isdigit():
            chosen_idx = int(user_choice) - 1
            if 0 <= chosen_idx < len(recommended):
                chosen_service = recommended.iloc[chosen_idx]
                
                # Pulire l'HTML e mostrare la descrizione, il campo "how_to" e i link dei moduli
                description_clean = clean_html(chosen_service['description'])
                how_to_clean = clean_html(chosen_service['how_to'])
                
                print(f"\nChatbot: Maggiori informazioni su '{chosen_service['name']}':\n")
                print(f"Descrizione: {description_clean}")
                print(f"Come accedere al servizio: {how_to_clean}")
                
                # Mostra i link dei moduli, se disponibili
                if chosen_service['module_links']:
                    print("\nModuli disponibili:")
                    for idx, link in enumerate(chosen_service['module_links'], 1):
                        print(f"{idx}. {link}")
                else:
                    print("\nNon sono disponibili moduli online per questo servizio.")
            else:
                print("Chatbot: Numero non valido.")
        else:
            print("Chatbot: Grazie per aver usato il nostro servizio. A presto!")

# Avvia il chatbot
chatbot()



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Chatbot: Benvenuto! Sono qui per aiutarti a trovare il servizio pubblico che stai cercando.


Chatbot: Per favore, dimmi cosa stai cercando (puoi inserire una parola chiave o una frase):  biomassa


Chatbot: Ecco i servizi che ho trovato per 'biomassa':
1. MONITORAGGIO DELL'UTILIZZO DI BIOMASSA
2. Contributo integrativo alla locazione sul libero mercato
3. RSA e altri servizi socio sanitari: svincolo, locazione o alienazione immobile
4. Contributi per i danni causati dalla calamità del 28 e 29 luglio 2024 - veicoli
5. Contributi per i danni causati dalla calamità del 28 e 29 luglio 2024 - immobili


Chatbot: Vuoi maggiori informazioni su uno di questi servizi? Inserisci il numero corrispondente oppure 'no':  1



Chatbot: Maggiori informazioni su 'MONITORAGGIO DELL'UTILIZZO DI BIOMASSA':

Descrizione: L’energia prodotta da biomasse, ovvero dall’insieme dei prodotti organici vegetali e animali utilizzati a fini energetici, rappresenta una delle principali fonti di energia rinnovabile attualmente consumata nell’UE e, in prospettiva degli obiettivi 2020, acquisirà ancora maggiore rilevanza. Anche in provincia di Trento, le biomasse rappresentano una quota significativa di consumi, dopo l’energia idroelettrica la fonte rinnovabile più utilizzata. Il legno da foresta, inteso nei suoi più vari sottoprodotti (residui dalle operazioni di taglio, scarti di prima lavorazione, legna da ardere) è senza dubbio l’elemento principale.Le biomasse sono delle fonti energetiche, prodotte e utilizzate a scala locale, che rappresentano una risorsa dai molteplici e positivi aspetti: da una parte, costituiscono un elemento indispensabile per far fronte alla crescente dipendenza delle importazioni in campo energetico

### TEST 3 - Codice per creare un riassunto automatico delle descrizioni dei servizi:


Chatbot: Benvenuto! Sono qui per aiutarti a trovare il servizio pubblico che stai cercando.
Chatbot: Per favore, dimmi cosa stai cercando (puoi inserire una parola chiave o una frase): avviare un'attività

Nome del servizio: Contributi per nuove imprese
Riepilogo: Questo servizio permette di ottenere agevolazioni per l'avvio di nuove imprese. I requisiti includono la presentazione di un business plan e una dichiarazione di intenti. I finanziamenti possono coprire fino al 50% dei costi.
Come accedere al servizio: Puoi richiedere il contributo compilando il modulo online e presentando la documentazione necessaria.
Moduli disponibili: https://provincia.example.com/moduli/nuova-impresa
----------------------------------------

Nome del servizio: Registrazione fiscale per nuove aziende
Riepilogo: La registrazione fiscale è obbligatoria per tutte le nuove aziende. Include l'apertura di una partita IVA e la registrazione presso l'Agenzia delle Entrate.
Come accedere al servizio: Il servizio è accessibile tramite il portale dell'Agenzia delle Entrate con SPID o CNS.
Moduli disponibili: https://provincia.example.com/moduli/registrazione-fiscale
----------------------------------------



In [43]:
import requests
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from transformers import pipeline, BartTokenizer  # Summarization con Transformers

# Scarica la lista di stop words italiane (se non lo hai già fatto)
nltk.download('stopwords')
stop_words_italian = stopwords.words('italian')

# Step 1: Scarica i dati dal link fornito
url = 'https://www.provincia.tn.it/api/openapi/servizi'
response = requests.get(url)

# Step 2: Verifica il tipo di dati e carica il JSON
try:
    data = response.json()  # Converte la risposta in JSON
except json.JSONDecodeError:
    print("Errore nella decodifica del JSON")
    data = None

# Controlla che i dati siano nel formato atteso
if 'items' in data:
    services_data = data['items']
else:
    print("Formato JSON non valido. Impossibile trovare la chiave 'items'.")
    services_data = []

# Step 3: Estrarre le informazioni rilevanti (id, nome, descrizione, how_to, has_module_input)
services = []
for item in services_data:
    modules = item.get('has_module_input', [])
    module_links = [module.get('uri', '') for module in modules if module.get('uri')]

    services.append({
        'id': item.get('id'),
        'name': item.get('name', 'Nome non disponibile').strip(),
        'description': item.get('description', 'Descrizione non disponibile'),
        'how_to': item.get('how_to', 'Informazioni non disponibili'),
        'module_links': module_links
    })

# Creare un DataFrame
df = pd.DataFrame(services)

# Funzione per convertire l'HTML in plain text
def clean_html(raw_html):
    soup = BeautifulSoup(raw_html, 'html.parser')
    return soup.get_text()

# Funzione per riassumere il testo usando Hugging Face Transformers con controlli sulla lunghezza
def summarize_text(text, max_length=100):
    summarizer = pipeline('summarization', model="facebook/bart-large-cnn", device=-1)  # Forza l'uso della CPU
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")  # Tokenizzatore
    
    # Tokenizza il testo e verifica se supera i limiti
    tokens = tokenizer.encode(text, return_tensors='pt')
    
    if len(tokens[0]) < 50:  # Se il testo è troppo corto
        return "Il testo è troppo breve per essere riassunto."
    elif len(tokens[0]) > 512:  # Limite ridotto a 512 token
        text = tokenizer.decode(tokens[0][:512], skip_special_tokens=True)  # Limita a 512 token
    
    # Genera il riepilogo
    try:
        summary = summarizer(text, max_length=max_length, min_length=30, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        return f"Errore durante il riepilogo: {str(e)}"

# Step 4: Funzione di raccomandazione e sintesi basata su matching parziale del testo inserito
def recommend_services_and_summarize(service_query='Contributo integrativo alla locazione sul libero mercato', top_n=5):
    # Step 4a: Normalizza il testo inserito dall'utente (lowercase e spazi rimossi)
    service_query = service_query.strip().lower()
    
    # Step 4b: Concatena nome e descrizione dei servizi per il matching
    df['combined_text'] = df['name'].str.lower() + " " + df['description'].str.lower()
    
    # Step 5: Calcolo della matrice TF-IDF sulla combinazione di nome e descrizione dei servizi
    tfidf = TfidfVectorizer(stop_words=stop_words_italian)
    tfidf_matrix = tfidf.fit_transform(df['combined_text'])
    
    # Step 6: Calcolo della similarità del coseno tra l'input dell'utente e tutti i servizi
    user_input_tfidf = tfidf.transform([service_query])
    cosine_sim = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()
    
    # Step 7: Prendi i top N servizi ordinati per similarità
    sim_scores = list(enumerate(cosine_sim))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:top_n]
    
    # Recupera i nomi dei servizi raccomandati
    service_indices = [i[0] for i in sim_scores]
    recommended_services = df.iloc[service_indices][['name', 'description', 'how_to', 'module_links']]
    
    # Crea un riepilogo per ogni servizio
    for idx, service in recommended_services.iterrows():
        description_clean = clean_html(service['description'])
        summary = summarize_text(description_clean, max_length=100)  # Riassume la descrizione
        print(f"\n\nNome del servizio: {service['name']}")
        print(f"\nRiepilogo: {summary}")
        print(f"\nCome accedere al servizio: {clean_html(service['how_to'])}")
        if service['module_links']:
            print(f"\nModuli disponibili: {', '.join(service['module_links'])}")
        print("-" * 40)

# Funzione per simulare il chatbot e fornire sintesi personalizzate
def chatbot():
    print("Chatbot: Benvenuto! Sono qui per aiutarti a trovare il servizio pubblico che stai cercando.")
    
    # Fase 1: Chiedi all'utente cosa sta cercando
    user_input = input("Chatbot: Per favore, dimmi cosa stai cercando (puoi inserire una parola chiave o una frase): ")
    
    # Fase 2: Trova servizi pertinenti e fornisci il riepilogo
    recommend_services_and_summarize(user_input, top_n=5)

# Avvia il chatbot
chatbot()



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Chatbot: Benvenuto! Sono qui per aiutarti a trovare il servizio pubblico che stai cercando.


Chatbot: Per favore, dimmi cosa stai cercando (puoi inserire una parola chiave o una frase):  biomassa




Nome del servizio: MONITORAGGIO DELL'UTILIZZO DI BIOMASSA

Riepilogo: L’energia prodotta da biomasse, ovvero dall’insieme dei prodotti organici vegetali e animali utilizzati a fini energetici, rappresenta una delle principali fonti di energia rinnovabile. In prospettiva degli obiettivi 2020, acquisirà ancora maggiore rilevanza. Anche in provinc

Come accedere al servizio: Il sistema prevede l'accesso ad un sistema online per i tutti i gestori di impianti di teleriscaldamento del Trentino tramite credenziali (utente e password) che vengono fornite da APRIE tramite richiesta al'indirizzo aprie@pec.provincia.tn.it.

Moduli disponibili: https://www.provincia.tn.it/api/openapi/documenti-e-dati/modulistica/c25565b09a2ca221a153424a07c7194b#Guida-alla-compilazione, https://www.provincia.tn.it/api/openapi/documenti-e-dati/modulistica/5e1764335426331737b78bfc56f72cbc#Copia-esempio-del-questionario
----------------------------------------


Nome del servizio: Contributo integrativo alla locazio